# Práctica 3 : Funciones de un solo sentido

## Ejercicio 1

Sea $(a_1, ... , ak)$ una secuencia super-creciente de números positivos (la suma de los términos que preceden a $a_i$ es menor que $a_i$, para todo $i$). Elige $n > \sum{a_i}$, y $u$ un entero positivo tal que $gcd(n,u) = 1$. Define $a_i^∗=u a_i \pmod {n}$. La función mochila (knapsack) asociada a $(a_1^∗, ... , a_k^∗)$ es

$$ f : \mathbb{Z}_2^k \rightarrow \mathbb{N},\ f(x_1, ... , x_k) = \displaystyle\sum_{i=1}^k x_i a_i^* $$

Implementa esta función y su inversa, tal y como se explica en _[2, §4.5]_. La llave pública es $(a_1^∗, ... , a_k^∗)$, mientras que la privada (y puerta de atrás) es $((a_1, ... , a_k), n, u)$.

Para realizar este ejercicio, nos basamos en [4.5.The knapsack cipher](http://www.maths.qmul.ac.uk/~pjc/notes/crypt.pdf).

En este ejercicio se nos pide implementar un cifrado de mochila de clave asimétrica donde la clave privada es una lista super-creciente y dos enteros $n$ y $u$ primos entre sí. La clave pública es una versión modificada de la lista super-creciente obtenida multiplicando cada elemento de la lista supercreciente por $u \pmod n$.

En el **problema de la mochila**, se nos da una mochila con un volumen $b$ y unidades $a_1, a_2, ..., a_k$. Queremos saber si podemos llenar la mochila usando algunos de las unidades. Los datos de entrada de este problema consisten en el número $b$ y la lista $(a1, a2, ..., ak)$ de números. 

La sucesión $(a_1, a_2, ..., a_k)$ de enteros positivos se llama super-creciente si cada término es mayor que la suma de sus predecesores, entonces si los datos del problema de la mochila son super-crecientes, entonces el algoritmo voraz está garantizado para resolver el problema. 

#### Algoritmo Greedy:

**Entrada**:
 - **n**: número del cual quiere conocerse sus sumandos en la lista $s$
 - **lista**: secuencia super-creciente
  
**Salida**:
 - **out**: lista que contiene los elementos de $s$ que al sumar dan $n$

In [1]:
# Función Greedy
# --------------------------------------------------------------------------------------------
def alg_greedy(n, lista):
    
    l = lista[:] # Realizamos una copia profunda de nuestra secuencia
    suma = 0 # Variable que suma los valores cogidos de la lista
    out = [] # Variable de salida
    
    # Mientras la suma sea distinta al elemento y no sobrepasemos la longitud de la secuencia
    while (suma != n) and (len(l)>0):
        
        # Cogemos el valor mayor de la lista
        aux = max(l)
        
        # Comenzamos a obtener los sumandos de la lista para n 
        # Si ese valor cogido es menor que nuestro número
        if (suma + aux) <= n:
            
            # Lo añadimos a la lista
            out.append(lista.index(aux))
            
            # Y sumamos ese valor 
            suma += aux            
           
        # Lo elimamos de la lista
        l.remove(aux)
        
    # Devolvemos la lista final
    return out

Para **cifrar**, cada bit del mensaje se multiplica por un elemento de la lista super-creciente (que es la llave pública) y se suma cada uno de estos productos, obteniéndose un número. 

La función de cifrado, es la sumatoria del producto de cada bit del mensaje por cada elemento de la llave pública. La clave pública consiste en una k-tupla $(a_1, a_2, ..., a_k)$ de enteros. 

#### Cifrado de Knapstack:

**Entrada**:
 - **n** y **u**: numeros primos entre sí
 - **lista**: secuencia super-creciente
 - **m**: mensaje a cifrar formado por lista binaria de 1 y 0
  
**Salida**:
 - **c**: mensaje cifrado

In [2]:
import fun_auxiliares as aux

# Función Cifrado de Knapstack
# --------------------------------------------------------------------------------------------
def knapsack(n, u, lista, m):
        
    # Condiciones que deben cumplirse siempre
    # Condición 1: Que n sea más grande que la suma de nuestra secuencia super-creciente 
    assert(n > sum(lista))
    # Condición 2: Que n y u sean primos relativos entre sí (mcd(n,u) = 1)
    assert(aux.alg_euclides(n, u)[4] == 1)
    
    # Obtenemos llave pública
    # Secuencia obtenida a partir de una super-creciente [a1*...ak*] haciendo "u·a[i] mod n"
    llave = [(a*u)%n for a in lista] 
    
    # Obtenemos el mensaje cifrado
    c = sum([e*a for e,a in zip(m, llave)])
    
    # Devolvemos la llave y el mensaje cifrado
    return llave, c

Para descifar, se calcula la sumatoria del producto de cada bit del mensaje con la lista super-creciente y descomponemos este número como suma de los elementos de la lista super-creciente.

Por lo que calcularemos el inverso $v$ de $u \pmod n$ usando el Algoritmo de Euclides. Luego calcularemos $b = vb^* \pmod n$ y tenemos:

$b = vb^* \pmod n \longrightarrow$ $b^*$ es el mensaje cifrado  
$b = v \cdot \sum e_i a_i^* \longrightarrow$ $a_i^*$ es la lista super-creciente modificada y $e_i$ el mensaje a cifrar  
$b = v \cdot \sum e_i(ua_i) \pmod n \longrightarrow$ $a_i$ es la lista super-creciente  
$b = (uv) \cdot \sum e_i a_i$  
$b = \sum e_i a_i \pmod n$  

Al final el descifrado se convierte en buscar los sumandos de $b$ en la lista supercreciente.

#### Inversa de Knapstack:

**Entrada**:
 - **n** y **u**: numeros primos entre sí
 - **lista**: secuencia super-creciente
 - **c**: mensaje cifrado
  
**Salida**:
 - **out**: mensaje descifrado (lista binaria)

In [3]:
import fun_auxiliares as aux

# Función Descifrado de Knapstack (Inversa de Knapstack)
# --------------------------------------------------------------------------------------------
def inversa_knapsack(n, u, lista, c):
    
    # Calculamos el inverso
    inverso = aux.alg_inverso(u, n)
    l_orig = [(a*inverso) % n for a in lista]
    b = (c*inverso) % n
    
    # Descomponemos b como suma de elementos de la mochila
    indices = alg_greedy(b, l_orig)
    
    # Expresamos los elementos de la mochila como binario
    out = [0 for i in l_orig] # Establecemos todos a cero 
    
    # Si el elemento de lista está en la mochila, ponemos 1
    for i in indices: 
        out[i] = 1
        
    # Devolvemos el mensaje descifrado
    return out

Realizaremos un pequeño ejemplo:

In [4]:
# Secuencia super-creciente
s = [1, 3, 7, 15, 31, 63, 127, 255]

# Escojamos dos números primos entre sí
n = 557
u = 323

# Nuestro mensaje
m = [0,1,1,0,0,1,0,1] # 1228
print("Mensaje: ", m)

# Ciframos
llave, c = knapsack(n, u, s, m)
print("Mensaje cifrado:", c)
print("Llave:", llave)

# Desciframos nuestro mensaje
d = inversa_knapsack(n, u, llave, c)
print("Mensaje descifrado:", d)

Mensaje:  [0, 1, 1, 0, 0, 1, 0, 1]
Mensaje cifrado: 1228
Llave: [323, 412, 33, 389, 544, 297, 360, 486]
Mensaje descifrado: [0, 1, 1, 0, 0, 1, 0, 1]


----

## Ejercicio 2

Sea $p$ un (pseudo-)primo mayor o igual que vuestro número de identidad. Encuentra un elemento primitivo, $\alpha$, de $\mathbb{Z}_p^*$ (se puede usar _[3, 2.132 (iv)]_; para facilitar el criterio, es bueno escoger $p$ de forma que $(p − 1)/2$ sea también primo, y para ello usamos Miller-Rabin). Definimos

$$ f : \mathbb{Z}_p \rightarrow \mathbb{Z}_p, x \rightarrow \alpha $$

Calcula el inverso de tu fecha de nacimiento con el formato AAAAMMDD.

Lo que nos pide es el **Logaritmo discreto**. Para ello, haremos uso de las funciones $alg\_miller\_rabin1()$, $jacobi()$ y $paso\_enano\_paso\_gigante()$.

Por tanto, ya estamos en disposición de implementar lo que nos piden en el ejercicio. La función busca un elemento primitivo alfa de $\mathbb{Z}_p^*$, con $p$ el primer primo mayor que mi DNI. Un elemento $\alpha$ será primitivo si el símbolo de jacobi de $\alpha$ sobre $p$ es $-1$. Después, se calcula la inversa de mi fecha de nacimiento usando el logaritmo discreto, que es la inversa de la función que se propone.

Primero vamos a obtener el siguiente primo mayor que mi DNI:

- Mi DNI: 75572158
- Siguiente primo mayor: 75572183

> Para obtener el siguiente primo mayor que mi DNI, he usado en GAP, la siguiente función _NextPrimeInt(75572158)_

#### Pseudocódigo

**Entrada**:
- **fecha**: El valor de la fecha de nacimiento en formato AAAMMDD
    
**Salida**: 
- El inverso de la fecha de nacimiento

In [5]:
import fun_auxiliares as aux

# Función que encuentra un elemento primitivo -> Calcula el inverso de mi fecha de nacimiento
# --------------------------------------------------------------------------------------------
def primitivo(fecha):

    # Encuentra un elemento primitivo alfa de Zp, n es fecha de nacimiento
    
    # Buscamos un pseudoprimo mayor o igual que mi DNI
    p = 75572158
    primo = False
    
    # Mientro no sea primo
    while not primo:
        
        # Comprobar que 'p' y '(p-1)/2' son primos
        if ( (aux.alg_miller_rabin1(p,10)) and (aux.alg_miller_rabin1((p-1)/2, 10)) ):
            primo = True
        
        # Sino, sumamos uno a 'p'
        else:
            p = p+1
    
    # Búsqueda del elemento primitivo    
    primitivo = False
    alfa = 2
            
    # Mientras no sea primitivo        
    while not primitivo:

        # Comprobamos con Jacobi si alfa es primitivo
        if (aux.jacobi(alfa,p) == -1):
            primitivo = True
        
        # Sino, sumamos uno a alfa
        else:
            alfa = alfa +1
    
    # La inversa de la función es el logaritmo discreto
    inv = aux.paso_enano_paso_gigante(alfa, fecha, p)
            
    # Devolvemos la inversa, el alfa y nuestro número primo
    return inv, alfa, p

Hagamos un ejemplo, para ello introducimos nuestra fecha de nacimiento con el formato AAAAMMDD.

In [6]:
fecha = 19952508
inv,alfa,p = primitivo(fecha)
print (inv,alfa,p)

11352288 5 75572423


El primer número ($inv$) es el inverso, el segundo ($alfa$) el alfa primitivo y el tercero ($p$) el primo. Para comprobar que está bien, solo hay que aplicar la función al resultado, es decir, hacer $alfa^{11352288} \pmod p$

In [7]:
print (aux.alg_potencia(alfa,inv,p))

19952508


---

**En lo que sigue, $p$ y $q$ son enteros primos, y $n = pq$**

## Ejercicio 3

Sea $f : \mathbb{Z}_n \rightarrow \mathbb{Z}_n$ la función de Rabin: $f(x) = x^2$. Sea $n = 48478872564493742276963$. Sabemos que $f(12) = 144 = f (37659670402359614687722)$. Usando esta información, calcula $p$ y $q$ (mira la demostración de _[1, Lemma 2.43]_).

En este ejercicio se nos pide que hagamos la descomposición en primos de un número.

In [8]:
import fun_auxiliares as aux

# Función que realiza una descomposición en primos
# --------------------------------------------------------------------------------------------
def alg_descomposicion(n, f1, f2):
    
    # 37659670402359614687722 + 12
    r1 = f1 + f2

    # 37659670402359614687722 - 12
    r2 = abs(f1-f2)
    
    # Calculamos el mcd de r1 y n
    mcd1 = aux.mcd(r1, n)
    
    # Si es distinto de 1, hemos terminado
    if mcd1 != 1:
        return (mcd1, n//mcd1)
    
    else:
        
        # Calculamos el mcd de r2 y n
        mcd2 = aux.mcd(r2, n)
        
        # Si es distinto de 1, hemos terminado
        if mcd2 != 1:
            return (mcd2, n//mcd2)
    
    # Sino ha entrado en el IF de antes, comprobamos que nuestro n es primo
    if aux.alg_miller_rabin1(n, 10):
        return (n, 1)

Vamos hacer un ejemplo. Para ello pasamo como entrada:

- $n=48478872564493742276963$

Además, sabemos que  $f(12) = 144 = f(37659670402359614687722)$

- $f1 = 12$
- $f2 = 37659670402359614687722$

In [9]:
n = 48478872564493742276963
f1 = 12
f2 = 37659670402359614687722

d = alg_descomposicion(n, f1, f2)

print("Número: ", n)
print("Descomposicion de n: ", d)

# Vamos a comprobar que devuelven el número original (de que son divisores de n)
print("Multiplicación de la descomposición de n:", d[0]*d[1])

Número:  48478872564493742276963
Descomposicion de n:  (303948303229, 159497098847)
Multiplicación de la descomposición de n: 48478872564493742276963


---

## Ejercicio 4

Elige $a_0$ y $a_1$ dos cuadrados arbitrarios módulo $n$ ($n$ como en el _Ejercicio 3_). Sea 

$$ h : \mathbb{Z}_2 × (\mathbb{Z}_n)^* \rightarrow (\mathbb{Z}_n)^*, h(b,x) = x^2 a_0^b a_1^{1-b} $$
    
Usa la construcción de Merkle-Damgard para implementar una función resumen tomando $h$ como función de compresión (esta $h$ fue definida por Goldwasser, Micali y Rivest). Los parámetros $a_0$, $a_1$ y $n$ se hacen públicos (la función debería admitir un parámetro en el que venga especificado el vector inicial).

Una **función resumen** (o función hash) es una función de una sola dirección resistente a colisiones. Estas funciones transforman cada posible mensaje en un resumen de un tamaño determinado (por ejemplo, $128$ ó $160$ bits). 

Las condiciones que tiene que satisfacer una función resumen son:

1. _Facilidad de cálculo._ Dado un mensaje $m$, el cálculo de $y = h(m)$ es sencillo y rápido.
2. _Unidireccionalidad._ Dado un resumen $h(m)$ es computacionalmente muy costoso encontrar $m$ tal que $y = h(x)$.
3. _Difusión._ El resumen debe ser función de todos los bits del mensaje. Un cambio en un bit debe afectar a aproximadamente la mitad de los bits del resumen.
4. _Compresión._ El resumen debe tener longitud fija.
5. _Resistencia débil a colisiones._ Dado $m$ es computacionalmente intratable encontrar $m' != m$ tal que $h(m) = h(m')$
6. _Resistencia fuerte a colisiones._ Es computacionalmente inviable encontrar $m$ y $m'$, distintos, tales que $h(m) = h(m')$.

In [10]:
import random
import fun_auxiliares as aux

# Función h, tomada como compresión
# --------------------------------------------------------------------------------------------
def funcion_h(b, x):
    
    # Obtenemos n (el del ejercicio 3)
    n = 48478872564493742276963
    
    # Elegimos a0 y a1 dos cuadrados arbitrarios mod n
    #a0 = (6347823468234**2) % n
    a0 = (random.randint(1, n-1) ** 2) % n
    #a1 = (5143151387122**2) % n
    a1 = (random.randint(1, n-1) ** 2) % n
    
    # Calculamos la función
    salida = ( aux.alg_potencia(x,2,n) * aux.alg_potencia(a0,b,n) * aux.alg_potencia(a1,1-b,n) )
    
    salida = salida % n
    
    return salida

Ahora, vamos a proceder a calcular la función resumen, que usa una estructura de Merkle-Damgard usando h como función de compresión.

- **Entrada**: Función de compresión (h).

- **Salida:** Funcion resumen.

La función empezará calculando $h$, para un bit principal $x$, para los sucesivos bits se usa el resultado obtenido en el anterior bit. Cuando hemos recorrido nuestro mensaje (bits), se devuelve el resultado del último bit.

Para ello, haremos uso de la función implementada en la P2, que convierte un string a bits.

In [11]:
# Funcion que convierte un string a bits
# --------------------------------------------------------------------------------------
def str_to_binlist(message):
    
    # get a binary list
    binary = list(bin(int.from_bytes(message.encode('utf-8'), byteorder='big')))
    
    # return the stream of 0s and 1s
    binary.remove('b')
    
    return list(map(lambda x: int(x), binary))

In [12]:
# Función resumen, tomada como compresión
# --------------------------------------------------------------------------------------------
def funcion_resumen(x, m):
    
    # x vector inicial
    # m mensaje al que aplicar el hash

    # Pasamos nuestro mensaje a binario
    m_bin = str_to_binlist(m) # Función implementada en la P2
    
    salida = 0
    
    for i in range(0, len(m_bin)):
        
        # Construccion Merkle-Damgard
        # La salida de la h se combina en una nueva h con el nuevo bloque de mensaje que es m[i]
        salida = funcion_h(m_bin[i], x) # m[i] es b
        
        # Se sustituye al vector inicial en los siguientes pasos 
        x = salida 
        
    # salida_bin = bin(salida)    
    return salida

Vamos a probar un pequeño ejemplo, en donde nuestra llave es $7687$ y nuestro mensaje "hola".

In [13]:
clave = 7687
mensaje = "hola"

# Obtenemos la función resumen
funcion_resumen(clave, mensaje)

33951221760290832876277

---

## Ejercicio 5

Sea $p$ el menor primo entero mayor o igual que tu número de identidad, y sea $q$ el primer primo mayor o igual que tu fecha de nacimiento (AAAAMMDD). Selecciona $e$ tal que $gcd(e,(p−1)(q−1)) = 1$. Define la función RSA

$$ f : \mathbb{Z}_n \rightarrow \mathbb{Z}_n, x \rightarrow x^e $$

Calcula el inverso de 1234567890.

La fortaleza del **RSA** se basa en el problema de la factorización de números enteros como producto de primos.
Lo primero que hay que hacer es elegir las claves. Como en todo sistema de cifrado asimétrico, las claves van por parejas: pública y privada.

1. Para elegir la pareja de claves, lo primero que necesitamos es elegir dos primos grandes $p$ y $q$, y calcular su producto $n = p \cdot q$.
2. Ahora se elige un entero $e$ tal que $mcd(e,\phi(n)) = 1$. Recordemos que $\phi(n) = (p-1)\cdot(q-1)$. Ambos valores, $(n, e)$ constituyen la clave pública del criptosistema. El número $n$ se conoce como módulo del Criptosistema.
3. La clave privada es un entero $d$ tal que $d \cdot e = 1 \pmod {\phi(n)}$.

#### Pseudocódigo Inverso RSA

**Entrada:**
- $p$: numero de identidad
- $q$: fecha de nacimiento
- $y$: el valor al que calcular el inverso

**Salida:**
- $x$: el inverso de la función RSA para el valor $y$

Para mecanizar el proceso, nos creamos una función que calcula el siguiente primo, a partir de un número.

In [14]:
import fun_auxiliares as aux

# Función que calcula el siguiente primo
# --------------------------------------------------------------------------------------------
def NextPrime(p):
    
    if p%2==0:
        p=p+1
    
    while (not aux.alg_miller_rabin(p)):
        p+=2
    
    return p

In [15]:
import random
import fun_auxiliares as aux

# Función Inverso de RSA
# --------------------------------------------------------------------------------------------
def RSA_Inverso (DNI, fecha, y):
    
    # Sea p el menor primo entero mayor o igual que mi DNI
    p = NextPrime(DNI)
    # Sea q el menor primo entero mayor o igual que mi DNI
    q = NextPrime(fecha)
    
    # Calculamo phi
    phi_n = (p-1)*(q-1)
    
    # Obtenemos n como la multiplicación de p y q
    n = p*q
    
    # Damos un valor por defecto a 'e'
    e = 2
    
    # Elegimos un número 'e' que sea primo relativo con phi_n, sino aumentamos uno 'e'
    while(aux.mcd(e,phi_n) != 1):
        e = e + 1
        # e = random.randint(2, p)
    
    # print("\nLa clave pública es ", n, "," , e)
    
    # Para hallar la clave privada resolvemos la congruencia e·d = 1 (mod phi_n)
    # es decir, calculamos el inverso de 'e' en Z(phi_n)
    d = aux.alg_inverso(e, phi_n)
    
    # Inverso de x^e = y --> es sacar x dado y,e
    x = aux.alg_potencia(y, d, n)

    return x,e,n

In [16]:
p = 75572158
q = 19952508

# Elemento al que calcular el inverso
y = 1234567890 

# Calculamos el inverso
print (RSA_Inverso(p,q,y))

(532151422460233, 3, 1507855417178977)


Vamos a comprobar que nos devuelve el elemento original:

In [17]:
import fun_auxiliares as aux

print(aux.alg_potencia(532151422460233, 3, 1507855417178977))

1234567890


#### Interpretación del resultado obtenido 

1. Sea $p = 75572158$ y $q = 19952519$. Estos valores se mantienen en secreto. Calculamos $n$ que vale $75572183 \cdot 19952519 = 1507854918366002$. En tal caso se tiene que $\phi(n) = (p-1) \cdot (q-1) = (75572158-1) \cdot (19952519-1) = 1507855321654276$.
2. Elegimos un número $e$ que sea primo relativo con $1507855321654276$, por ejemplo $e = 3$. La clave pública es entonces $(1507854918366002, 3)$.
3. Para hallar la clave privada resolvemos la congruencia $3 \cdot d = 1(mód 1507855321654276)$, es decir, calculamos $3^{-1}$ en $\mathbb{Z}_{1507855321654276}$, que podemos hacerlo usando el algoritmo inverso de la P1. Dicho inverso vale 1005236881102851. La clave privada es entonces 1005236881102851.

---

## Ejercicio 6

Sea $n = 50000000385000000551$, y que sabemos que una inversa de $\mathbb{Z}_n \rightarrow \mathbb{Z}_n$, $x \rightarrow x^5$ es $x \rightarrow x^{10000000074000000101}$ (esto es, conoces tanto la llave pública como la privada de la función RSA). Encuentra p y q usando el método explicado en _[2, Page 92]_. Compara este procedimiento con el algoritmo de Miller-Rabin y el Ejercicio 3.

Para realizar este ejercicio nos hemos basado en la referencia aportada en el enunciado en la [página 92](http://www.maths.qmul.ac.uk/~pjc/notes/crypt.pdf)

#### Pseudocódigo (Métodos para encontrar los factores primos de $N$,  dados $d$ y $e$)

1. Sea $d \cdot e-1 = 2^a \cdot b$, donde $b$ es impar. 
2. Elija un $x$ aleatorio con $0 < x < N$.
3. Si $gcd (x, N) != 1$, ya hemos encontrado un factor y podemos detenerlo.
4. Sino $y = x^b \pmod N$.
5. Si $y ≡ ± 1 \pmod N$, el algoritmo ha fallado.
6. Guardar el valor de $y$ ($z = y$). 
7. Ir reemplazando $y$ por $y^2 \pmod N$.
8. Si $y ≡ -1 \pmod N$, el algoritmo ha fallado.
9. Si $y ≡ 1 \pmod N$, entonces hemos encontrado $z$ tal que $z^2 ≡ 1 \pmod N)$ y $z !≡ ± 1 \pmod N$. 
10. Entonces $gcd (N, z+1)$ y $gcd (N, z-1)$ son los factores primos de $N$.

**Entrada:** 
- $n$: entero a factorizar
- $d$: llave pública de la función 
- $e$: llave privada de la función inversa 

**Salida:**
- $p$ y $q$: factores primos de n
- $False$: si no es capaz de encontrar los factores

In [18]:
import random
import fun_auxiliares as aux

# Métodos para encontrar los factores primos de un número, dados d y e
# --------------------------------------------------------------------------------------------
def factorizar_numero(n,d,e):

    # Sea d*e-1 = (2**a)*b --> ponemos que de1 = d*e-1, donde b es impar. 
    de1 = d*e - 1
    
    # Expresamos d*e-1 como (2**a)*b con b impar --> Calculamos a y b
    b = de1
    a = 0
    while (b%2 == 0):
        b = b//2
        a = a+1
    
    # Repetimos con distintos x hasta encontrar factores no triviales
    while True: 
        
        # Elija un x aleatorio con 0<x<N, es decir, entre 1 y N
        x = random.randint(1,n-1)
        
        # Calcular gcd(x, N)
        gcd = aux.alg_euclides_v2(x, n)[0]
        
        # Si es distinto de 1, ya hemos encontrado un factor y podemos parar
        if (gcd != 1):
            return gcd
        
        # Si gcd (x,N) = 1
        if (gcd == 1):
            
            # y = (x**b) mod n
            y = aux.alg_potencia(x,b,n)
    
        # Repetimos 'a' veces
        for i in range(a): 
            
            # Debemos guardar el primer valor de 'y'
            z = y
            
            # Reemplazar 'y' por (y**2) mod N
            y = aux.alg_potencia(y,2,n) 
            
            # Si y ≡ ± 1 (mod N), el algoritmo ha fallado
            if (y == n-1):
                return False # Error
            
            # Si y ≡ 1 (mod N), entonces hemos encontrado 'z' tal que 
            # z2 ≡ 1 (mod N) y z !≡ ± 1 (mod N).
            if (y == 1):

                # Entonces gcd (N, z+1) y gcd (N, z-1) son los factores primos de N
                p = aux.alg_euclides_v2(n,z+1)[0]
                q = aux.alg_euclides_v2(n,z-1)[0]

                # Acabamos solo si no son factores triviales
                if(p != n and p != 1):         
                    return p,q
                
    return False

Primero, haremos el ejemplo que nos proporciona la referencia, para así comprobar si está bien realizado el ejercicio.

In [19]:
factorizar_numero(589, 7 ,13)

(31, 19)

#### Interpretación del Resultado Obtenido

Supongamos que $N = 589$ y $\lambda(N) = 90$. Ahora $589 \pmod 90 = 49$. Tratando que $\phi(N) = 540$, obtenemos que los factores primos de $N$ son las raíces del polinomio cuadrático $x^2 - 50x + 589 = 0$,
así que $p, q = 25 ± \sqrt{625-589} = 25 ± 6 = 31,19$.

No hay necesidad de probar para el otro caso.

O supongamos, que $N = 589$ y que nuestro exponente privado correspondiente a $d = 7$ que es $e = 13$. Ahora $d \cdot (e-1) = 90 = 2 \cdot 45$. Aplicar el algoritmo con $x = 2$. Ya que $mcd(2,589) = 1$. Ahora $y = 2^{45} \pmod 589 = 94$. En la etapa siguiente, $z = 94$ e $y = z^{2} \pmod 589 = 1$. Así que los factores de $mcd(589, 95) = 19$ y $mcd(589, 93) = 31$.

A continuación, procedemos hacer el ejercicio propuesto en el enunciado.

In [20]:
# Definimos nuestros números
n = 50000000385000000551
d = 5
e = 10000000074000000101

# Aplicamos la función factorizar
factorizar_numero (n, d, e)

(5000000029, 10000000019)

In [21]:
import random
import fun_auxiliares as aux

# Métodos para encontrar los factores primos de un número (Versión 2)
# --------------------------------------------------------------------------------------------
def factorizar_numero_v2(n, d, e):
    
    assert(e!=0 and d!=0)
    
    factores = []
    
    a = 0
    
    de = d*e
    de = de-1
    
    a = aux.alg_potencia_v2(de)
    b = de//aux.alg_potencia(2, a, n)
    
    x = random.randint(1, n-1)

    divisor = aux.alg_euclides_v2(x, n)[0]

    if divisor!=1:
        return (divisor, n//divisor)

    y = aux.alg_potencia(x, b, n)
        
    r1 = (y-1)%n
    r2 = (y+1)%n
    
    if r1 == 0 and r2 == 0:
        return False
    
    while aux.alg_euclides_v2(y, n)[0]==1:
        z = y
        y = aux.alg_potencia(y, 2, n)
        
        if y == n-1:
            return False 
        
        if y==1:
            
            f1 = aux.alg_euclides_v2(n, z-1)[0]
            f2 = aux.alg_euclides_v2(n, z+1)[0]
            
            factores.append(f1)
            factores.append(f2)

            return factores
        
        
# Probemos un ejemplo        
factorizar_numero_v2(589, 7 ,13)

[1, 589]

**Comparación con el _ejercicio 3_**

Debemos tener en cuenta, que ambos algoritmos son equivalentes. Pero para realizar la comparación nos podemos basar en que el ejercicio 3 necesitamos para obtener los factores, el resultado de dos valores para la función _Rabin()_ o en mi caso _alg-descomposicion()_. Y sin embargo, para este ejercicio, solo hace falta obtener la inversa de la función para lograr la factorización (es decir, la llave pública y privada.)

_De normal, no hay acceso ni a la llave privada ni a la pública, aunque, es posible adquirir el resultado de dos valores para la función de Rabin() o en mi caso alg-descomposicion()._

---

## Ejercicio 7

En este ejercicio se pide implementar un sistema de firma digital y verificación de la firma. Se puede elegir entre firma RSA o DSS.

Al igual que antes, debe realizar tres tareas: generación de claves (ejercicios anteriores), generación de firma y verificación de firma.

Nos hemos decantado por la firma **RSA**:

Para la **generación de la clave**, nos basamos en la función generada en el _ejercicio 5_ que implementa un $RSA$.

1. Para elegir la pareja de claves, lo primero que necesitamos es elegir dos primos grandes $p$ y $q$, y calcular su producto $n = p \cdot q$.
2. Ahora se elige un entero $e$ tal que $mcd(e,\phi(n)) = 1$. Recordemos que $\phi(n) = (p-1)\cdot(q-1)$. Ambos valores, $(n, e)$ constituyen la clave pública del criptosistema. El número $n$ se conoce como módulo del Criptosistema.
3. La clave privada es un entero $d$ tal que $d \cdot e = 1 \pmod {\phi(n)}$.

#### Generación de claves RSA

Dichas claves, serán guardadas en un fichero.
    
- **Entrada**: 
    - o _p,q_ como dos primos grandes 
    - o que se generen ambos primos dentro de la función de manera aleatoria
    
- **Salida**:
    - _e,n_: clave publica
    - _d_: clave privada

In [22]:
import random
import fun_auxiliares as aux

# Método para generar una clave
# --------------------------------------------------------------------------------------------
def generacion_claves():

    # Por seguridad es conveniente elegir dos números de manera aleatoria
    p = random.randint(1, 1000000000000000)
    q = random.randint(1, 1000000000000000)
    
    # Si p no es primo, forzamos a que sea
    p = NextPrime(p)
    
    # Si q no es primo, forzamos a que sea
    q = NextPrime(q)
    
    # Realizamos su producto
    n = p*q
    
    # Calculamos phi
    phi = (p-1)*(q-1)
    
    e = 2
    # Elegimos un número 'e' que sea primo relativo con phi_n, sino aumentamos uno 'e'
    while(aux.alg_euclides_v2(phi,e)[0] != 1):
        e = e +1
    
    # Para hallar la clave privada resolvemos la congruencia e·d = 1 (mod phi_n)
    # es decir, calculamos el inverso de 'e' en Z(phi_n)
    d = aux.alg_inverso(e,phi)
    
    # Abrimos los ficheros donde se guardarán las claves (debe tener permiso de escritura)
    try:
        f_pub = open("clave_publica", "w")
        f_priv = open("clave_privada", "w")
    except:
        print("Error abriendo los ficheros de las claves.")
        
    # Escribimos en un fichero la clave publica y privada, separando e y d de n con un espacio
    # Clave Pública
    f_pub.write(str(e)+" ")
    f_pub.write(str(n))
    # Clave Privada
    f_priv.write(str(d)+" ")
    f_priv.write(str(n))
    
    # Devolvemos las claves
    return e, n, d

Para la **generación de la firma**, se le introducirá un mensaje a cifrar (fichero) y el fichero con la clave (privada), y deberá generar una firma, que se guardará en un fichero de texto.

Puesto que lo que realmente se firma no es el mensaje, sino un resumen del mensaje, hay que generar un resumen de dicho mensaje. Para esto emplearemos la función SHA1 (se pueden añadir otras funciones resumen). Cualquiera de las implementaciones de esta función que hay en la red puede ser usada.

_Para la función SHAI1_, usaremos la librería de Python **hashlib**, que ya implementa esa función.

#### Generación de firma RSA  

Como podemos ver en los apuntes de Miranda, para generar una firma, solo nos bastaría hacer $fir(r) = r^d \pmod n$. Por tanto, el mensaje firmado sería el par $(r, fir(r))$.
    
- **Entrada**:
    - _mensaje_: nombre del fichero del mensaje a firmar en string
    - _clave-privada_: nombre del fichero de la clave privada en string (contiene d y n)
        
- **Salida**:
    - _f_: firma del resumen del mensaje

In [23]:
import hashlib 
import fun_auxiliares as aux

# Método para generar la firma
# --------------------------------------------------------------------------------------------
def generacion_firma(mensaje, clave_privada):

    # Abrimos el fichero de mensaje
    try:
        myfile = open(mensaje, encoding='utf-8')
    except:
        print("Error abriendo el fichero del mensaje.")
    
    # Lo guardamos en una variable 
    m = myfile.read()  
    m = m.encode("utf-8")

    # Calculamos un resumen del mensaje
    r = hashlib.sha1(m).hexdigest()
    
    # Como la salida de 'r' es en hexadecimal,
    # la convertimos a entero base 10 para poder operar con el resumen
    r = int("0x"+r, 0)
    
    # Leemos la clave del fichero
    try:
        f_priv = open("clave_privada")
    except:
        print("Error abriendo el fichero de la clave privada.")
        
    clave = f_priv.read().split(" ")
    d = int(clave[0])
    n = int(clave[1])
    
    # Generamos la firma (firma del resumen)
    f = aux.alg_potencia(r,d,n)
    
    try:
        f_firma = open("firma","w")
    except:
        print("Error abriendo el fichero firma.")
        
    # Escribios la firma en el archivo    
    f_firma.write(str(f))
    
    # Devolvemos la firma
    return f

Para la **verificación de la firma**, se introduce el mensaje (fichero) que se ha firmado, un fichero con la firma (con el mismo formato que el generado en el apartado anterior) y un fichero con la clave (pública). Deberá responder si la firma es o no válida.

#### Verificación de firma RSA  

Como podemos ver en los apuntes de Miranda, para comprobar si la firma es válida, habría que hacer si $r = fir(r)^e \pmod n$.

- **Entrada**:
    - _clave-publica_: nombre del fichero de la clave publica en string (e y n)
    - _mensaje_: nombre del fichero del mensaje a verificar en string
    - _firma_: nombre del fichero de firma en string
    
- **Salida**:
    - _True_: si la firma es correcta
    - _False_: si la firma no es correcta

In [24]:
import hashlib 
import fun_auxiliares as aux

# Método para verificar la firma
# --------------------------------------------------------------------------------------------
def verificacion_firma(clave_publica, mensaje, firma):
   
    # Abrimos el fichero de mensaje
    try:
        myfile = open(mensaje, encoding='utf-8')
    except:
        print("Error abriendo el fichero del mensaje.")
    
    # Lo guardamos en una variable como string
    m = myfile.read()
    m = m.encode("utf-8")
    
    # Calculamos un resumen del mensaje
    r = hashlib.sha1(m).hexdigest()
    
    # Como la salida de 'r' es en hexadecimal,
    # la convertimos a entero base 10 para poder operar con el resumen
    r = int("0x"+r,0)
    
    # Leemos la clave del fichero
    try:
        f_pub = open("clave_publica")
    except:
        print("Error abriendo el fichero de la clave pública.")
    
    clave = f_pub.read().split(" ")
    e = int(clave[0])
    n = int(clave[1])
    
    # Leemos el fichero con la firma
    try:
        f_firma = open("firma")
    except:
        print("Error abriendo el fichero de la firma.")
    
    f = int(f_firma.read())
    
    # Verificamos la firma
    v = aux.alg_potencia(f,e,n)

    # Si la verificacion coincide con la función resumen es válidad
    if(v%n == r%n): 
        return True # Firma válida
    else:
        return False # Firma no válidad

In [25]:
# Nombre del fichero de la clave privada
clave_privada = "clave_privada"

# Nombre del fichero de la clave pública
clave_publica = "clave_publica"

# Nombre del fichero del mensaje
mensaje = "mensaje"

# Nombre del fichero de la firma
firma = "firma" 

# Hacemos la Firma RSA
generacion_claves()
generacion_firma(mensaje, clave_privada)
verificacion_firma(clave_publica, mensaje, firma)

True

Para comprobar que la verificación funciona, si editamos el fichero de firma y realizamos de nuevo la verificación, obtenemos un valor False.